In [1]:
import torch
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
from utils import *
delete_cache()

Deleting: __pycache__
All __pycache__ directories have been deleted.


In [2]:
# model_name = "./output/Qwen3-4B/checkpoint-75"
model_name = "Qwen/Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(
    model_name, 
    use_fast=False,
    trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto", 
    dtype=torch.bfloat16
)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
prompt = "你是一个香港保险经纪人，你在考试，你需要正确回答考试题目。回答格式举例（请注意，你的回答要由选项字母结尾，不要有多余的话）。理由：...（简单说一下理由即可）\n答案：E"

In [4]:
pred_list = []

with open("./val.jsonl", "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        line = line.strip()
        if not line:
            continue

        data = json.loads(line)
        question = data["question"]

        messages = [
            {"role": "system", "content": prompt},
            {"role": "user", "content": question}
        ]

        response = predict(messages, model, tokenizer)

        # 取 response 最后一个字符（防止空字符串）
        last_char = response[-1] if response else None
        pred_list.append(last_char)
        print(response)
        print(f"{idx+1}: {last_char}")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<think>
好的，我现在需要回答这个关于自然保费制度特点的题目。题目给出四个选项，我需要仔细分析每个选项是否正确，然后选择正确的答案。

首先，题目问的是自然保费制度的特点。自然保费制度通常指的是在保险期间，保费随着被保险人年龄增长而增加的制度。比如，寿险中的自然保费，随着年龄增长，死亡率上升，所以保费也会增加。这对应选项a)保費隨年齡增長而增加。这个应该是正确的。

接下来看选项b)會產生逆選擇。逆选择通常指的是高风险个体更倾向于购买保险，而低风险个体可能不买，这会导致保险公司面临更高的赔付风险。但自然保费制度本身是否会导致逆选择呢？可能需要进一步思考。自然保费制度可能通过年龄调整保费来部分应对风险，但逆选择更多是保险市场中的普遍问题，而不仅仅是自然保费制度的特点。所以b可能不是正确选项。

选项c)大部份保險公司已不採用。自然保费制度是否已经被大部分公司弃用？比如，现在很多保险公司使用均衡保费制度，即前期保费较低，后期较高，以平衡风险。自然保费可能在某些特定产品中仍有应用，但可能不普遍。所以c可能正确，但需要确认。

选项d)以上各項皆正確。如果a正确，而b和c是否正确呢？如果b和c都正确，那么d是对的。但需要判断每个选项是否正确。

现在需要确认自然保费制度的特点。自然保费制度的特点是保费随年龄增长而增加，这确实是其核心特点。而逆选择可能不是其直接特点，而是保险市场中的普遍问题，自然保费制度可能通过年龄调整保费来部分应对，但逆选择本身是否属于其特点呢？可能不是。至于选项c，是否大部分保险公司不再采用？可能自然保费在某些情况下仍然使用，比如长期寿险或某些特定产品，但可能不是大部分公司采用。因此，选项c可能不正确。

所以正确的应该是选项a，而选项d是否正确取决于其他选项是否正确。如果只有a正确，那么d不正确。因此，正确答案应该是a。但需要再仔细检查。

另外，自然保费制度是否会导致逆选择？可能不会直接导致，因为自然保费制度通过调整保费来应对年龄相关的风险，而逆选择更多是投保人选择保险时的道德风险问题。因此，选项b可能不正确。选项c是否正确？如果大部分保险公司已不采用，那么c正确，但需要确认。例如，自然保费可能在某些产品中仍然存在，比如终身寿险，但可能不是大部分公司采用。因此，选项c可能不正确。因此，正确答案是a，而选项d不正确。因此，正确答案是a。
</th

In [5]:
correct = 0
total = 0

with open("./val.jsonl", "r", encoding="utf-8") as f:
    for idx, line in enumerate(f):
        line = line.strip()
        if not line:
            continue

        data = json.loads(line)
        gold = data["answer"]  # 你已确认是 A / B / C / D

        pred = pred_list[idx]
        pred = pred.upper() if pred is not None else None

        total += 1
        if pred == gold:
            correct += 1

accuracy = correct / total if total > 0 else 0.0

print(f"Accuracy: {accuracy:.2f} ({correct}/{total})")

Accuracy: 0.72 (18/25)


In [6]:
pred_list

['A',
 'A',
 'A',
 'B',
 'C',
 'C',
 'D',
 'D',
 'D',
 'B',
 'B',
 'D',
 'A',
 'A',
 'C',
 'A',
 'C',
 'C',
 'C',
 'D',
 '个',
 'D',
 'C',
 'D',
 'A']

In [7]:
# test_texts = {
#     'instruction': prompt,
#     'input': "就人壽保險而言，判斷一項可保性的基本因素可能是a)準投保生命的性別\nb)準投保生命的年齡\nc)準投保生命的健康狀況\nd)較大的投保額"
# }

# instruction = test_texts['instruction']
# input_value = test_texts['input']

# messages = [
#     {"role": "system", "content": f"{instruction}"},
#     {"role": "user", "content": f"{input_value}"}
# ]

# response = predict(messages, model, tokenizer)
# print(response)